In [7]:
# Uninstall packages from Kaggle base image that are not needed.
%pip uninstall -qy jupyterlab jupyterlab-lsp
# Install the google-genai SDK for this codelab.
%pip install -qU 'google-genai==1.7.0'

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
from google import genai
from google.genai import types

from IPython.display import Markdown, HTML, display

genai.__version__

'1.7.0'

In [9]:
import os


GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
client = genai.Client(api_key=GOOGLE_API_KEY)


In [10]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [11]:


response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="When and where is Billie Eilish's next concert"
)

Markdown(response.text)


Unfortunately, Billie Eilish doesn't have any upcoming concerts scheduled that have been officially announced. The best places to stay up-to-date on any potential announcements are:

*   **Billie Eilish's Official Website:** This is the primary source for tour information.
*   **Billie Eilish's Social Media:** Check her official accounts on platforms like Instagram, X (formerly Twitter), and Facebook.
*   **Ticketmaster and other major ticketing platforms:** These will be updated as soon as any tour dates are released.

In [12]:
config_with_search = types.GenerateContentConfig(
    tools = [types.Tool(google_search=types.GoogleSearch(),)]
)

def query_with_grounding():
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents="When and where is billie eilish's next concert",
        config=config_with_search
    )

    return response.candidates[0]

rc = query_with_grounding()
Markdown(rc.content.parts[0].text)



Billie Eilish is currently on her "Hit Me Hard and Soft: The Tour". Based on available information, here are some of her upcoming concert dates and locations:

*   **April 23 & 24, 2025:** Avicii Arena, Stockholm, Sweden
*   **April 26, 2025:** Unity Arena, Fornebu, Norway
*   **April 28 & 29, 2025:** Royal Arena, Copenhagen, Denmark
*   **May 2, 2025:** ZAG Arena, Hannover, Germany
*   **May 4, 5 & 7, 2025:** Ziggo Dome, Amsterdam, Netherlands
*   **May 9, 2025:** Uber Arena, Berlin, Germany
*   **May 29 & 30, 2025:** LANXESS arena, Cologne, Germany
*   **June 1, 2025:** O2 arena, Berlin
*   **June 3 & 4, 2025:** TAURON Arena Kraków, Kraków, Poland
*   **June 6, 2025:** Wiener Stadthalle, Vienna, Austria
*   **June 8, 2025:** Unipol Arena, Casalecchio di Reno, Italy
*   **June 10 & 11, 2025:** Accor Arena, Paris, France
*   **June 14 & 15, 2025:** Palau Sant Jordi, Barcelona, Spain
*   **July 7 & 8, 2025:** OVO Hydro, Glasgow, UK
*   **July 10, 11, 13, 14, 16 & 17, 2025:** The O2 arena, London, UK
*   **July 19 & 20, 2025:** Co-op Live, Manchester, UK
*   **July 22 & 23, 2025:** Utilita Arena, Birmingham, UK
*   **July 26 & 27, 2025:** 3Arena, Dublin, Ireland



In [13]:
while not rc.grounding_metadata.grounding_supports or not rc.grounding_metadata.grounding_chunks:
    rc = query_with_grounding()

chunks = rc.grounding_metadata.grounding_chunks

for chunk in chunks:
    print(f'{chunk.web.title}: {chunk.web.uri}\n')
    



theo2.co.uk: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAIoO54HZZ6s9hD2GOfMs_hGJZguMeSHHEE5Goj93xgz5oKum8q0Te8q400owD-Wl5uTDUP6tr108MGMTnY1nqLdDOqnrQnqyAkU1pAdfC4rrwz4VMI1XCOctAk2noFnDp1XKlAwhjnmHQvyyJuwLR1xgtz85HUqXpiyt1jnFogYXW6afF00DPtzI3QMaA==

frontiertouring.com: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAI9uSCAO6XvjxvZqHoAf6UNvkvElKxD0RWZXpK19WJkwO2Qb0jcRz_kTZuWi4xAw_k8Fa_WNxYZ3EX86xyVjA90cu3ExI7YhJjg-2IJ5V2DAho15np9O-kMcXSzM1z-2X943ACG9Cu5

ticketmaster.co.uk: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqALLTuC6oL-j251gjJiznJ0D1uo7h3II1CUJlIt76hyTiIhouqGkp33G4hrehyx3vsdJD6D2aeQumwbVfz066-OHofMq3kepVKqury9X-blf5TuLRLmqks9XXwOLvYmDnMlFGXmhj8cyTalGiHVTEbYqw69tm4e3Lw63eBbnnec=

wikipedia.org: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqALaaLTmx8QfskKpoKwjrrgqK0Cr1tcrXg3yJMqZURcDuWU2y5AflUrq78_3ZdVvAqiUCjqyA28KAFEMRAf9Bo4ZnGy767umSfztwSiMqRK-rdMlzSEWHrA3pNel-nLtHDFeHe5JcJg26jLwg4G-ZR3fM

In [14]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)


In [15]:
from pprint import pprint
from re import L

supports = rc.grounding_metadata.grounding_supports

for support in supports:
    pprint(support.to_json_dict())

{'confidence_scores': [0.80626404,
                       0.88891524,
                       0.63002145,
                       0.77561367,
                       0.6613139,
                       0.939793],
 'grounding_chunk_indices': [0, 1, 2, 3, 4, 5],
 'segment': {'end_index': 67,
             'text': 'Billie Eilish is currently on her "Hit Me Hard and Soft: '
                     'The Tour".'}}
{'confidence_scores': [0.6104942],
 'grounding_chunk_indices': [4],
 'segment': {'end_index': 220,
             'start_index': 160,
             'text': '*   **April 23 & 24, 2025:** Avicii Arena, Stockholm, '
                     'Sweden'}}
{'confidence_scores': [0.83797866, 0.63206124],
 'grounding_chunk_indices': [6, 2],
 'segment': {'end_index': 273,
             'start_index': 221,
             'text': '*   **April 26, 2025:** Unity Arena, Fornebu, Norway'}}
{'confidence_scores': [0.6709798],
 'grounding_chunk_indices': [7],
 'segment': {'end_index': 335,
             'start_index': 27

In [16]:
import io
markdown_buffer = io.StringIO()

markdown_buffer.write("Supported text:\n\n")

for support in supports:
    markdown_buffer.write("*")
    markdown_buffer.write(
        rc.content.parts[0].text[support.segment.start_index:support.segment.end_index] 
    )

    for i in support.grounding_chunk_indices:
        chuck = chunks[i].web

        markdown_buffer.write(f'<sup>{i+1}</sup>')
    markdown_buffer.write("\n\n")

Markdown(markdown_buffer.getvalue())


Supported text:

*Billie Eilish is currently on her "Hit Me Hard and Soft: The Tour".<sup>1</sup><sup>2</sup><sup>3</sup><sup>4</sup><sup>5</sup><sup>6</sup>

**   **April 23 & 24, 2025:** Avicii Arena, Stockholm, Sweden<sup>5</sup>

**   **April 26, 2025:** Unity Arena, Fornebu, Norway<sup>7</sup><sup>3</sup>

**   **April 28 & 29, 2025:** Royal Arena, Copenhagen, Denmark<sup>8</sup>

**   **May 2, 2025:** ZAG Arena, Hannover, Germany<sup>8</sup><sup>3</sup>

**   **May 9, 2025:** Uber Arena, Berlin, Germany<sup>3</sup>

**   **May 29 & 30, 2025:** LANXESS arena, Cologne, Germany<sup>3</sup>

**   **June 1, 2025:** O2 arena, Berlin<sup>7</sup><sup>3</sup>

**   **June 3 & 4, 2025:** TAURON Arena Kraków, Kraków, Poland
*<sup>3</sup>

*  **June 6, 2025:** Wiener Stadthalle, Vienna, Austria
*<sup>3</sup>

*  **June 8, 2025:** Unipol Arena, Casalecchio di Reno, Italy
*<sup>3</sup>

*  **July 7 & 8, 2025:** OVO Hydro, Glasgow, UK
*<sup>3</sup>

*  **July 10, 11, 13, 14, 16 & 17, 2025:** The O2 arena, London, UK
*<sup>3</sup>

*  **July 19 & 20, 2025:** Co-op Live, Manchester, UK
*<sup>3</sup>

*  **July 22 & 23, 2025:** Utilita Arena, Birmingham, UK
*<sup>3</sup>

*  **July 26 & 27, 2025:** 3Arena, Dublin, Ireland

<sup>3</sup>



In [21]:
from IPython.display import display, Image, Markdown

def show_response(response):
    for p in response.candidates[0].content.parts:
        if p.text:
            display(Markdown(p.text))
        elif p.inline_data:
            display(Image(p.inline_data.data))
        else:
            print(p.to_json_dict())
    
        display(Markdown('----'))

In [18]:
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0
)

chat = client.chats.create(model='gemini-2.0-flash')

response = chat.send_message(
    message="Wha were the medals tallies, by to-10 countries, for 2024 olympics ?",
    config=config_with_search
)

show_response(response)

Here's the medal tally for the top 10 countries at the Paris 2024 Olympics:

1.  **United States of America:** 40 Gold, 44 Silver, 42 Bronze (126 total)
2.  **People's Republic of China:** 40 Gold, 27 Silver, 24 Bronze (91 total)
3.  **Japan:** 20 Gold, 12 Silver, 13 Bronze (45 total)
4.  **Australia:** 18 Gold, 19 Silver, 16 Bronze (53 total)
5.  **France:** 16 Gold, 26 Silver, 22 Bronze (64 total)
6.  **Netherlands:** 15 Gold, 7 Silver, 12 Bronze (34 total)
7.  **Great Britain:** 14 Gold, 22 Silver, 29 Bronze (65 total)
8.  **Republic of Korea:** 13 Gold, 9 Silver, 10 Bronze (32 total)
9.  **Italy:** 12 Gold, 13 Silver, 15 Bronze (40 total)
10. **Germany:** 12 Gold, 13 Silver, 8 Bronze (33 total)

-----

In [22]:
config_with_code = types.GenerateContentConfig(
    tools=[types.Tool(code_execution=types.ToolCodeExecution())],
    temperature=0.0,
)

response = chat.send_message(
    message="Now plot this as a seaborn chart. Break out the medals too.",
    config=config_with_code,
)

show_response(response)

Okay, I can create a Seaborn chart visualizing the medal tally for the top 10 countries at the 2024 Olympics. I'll use a stacked bar chart to represent the gold, silver, and bronze medals for each country.



----

{'executable_code': {'code': "import pandas as pd\nimport seaborn as sns\nimport matplotlib.pyplot as plt\n\n# Data for the medal tally\ndata = {\n    'Country': ['United States of America', 'People\\'s Republic of China', 'Japan', 'Australia', 'France', 'Netherlands', 'Great Britain', 'Republic of Korea', 'Italy', 'Germany'],\n    'Gold': [40, 40, 20, 18, 16, 15, 14, 13, 12, 12],\n    'Silver': [44, 27, 12, 19, 26, 7, 22, 9, 13, 13],\n    'Bronze': [42, 24, 13, 16, 22, 12, 29, 10, 15, 8]\n}\n\ndf = pd.DataFrame(data)\n\n# Melt the dataframe to long format for Seaborn\ndf_melted = df.melt(id_vars='Country', var_name='Medal', value_name='Count')\n\n# Create the stacked bar chart\nplt.figure(figsize=(12, 8))\nsns.barplot(x='Country', y='Count', hue='Medal', data=df_melted, palette=['#FFD700', '#C0C0C0', '#CD7F32']) # Gold, Silver, Bronze colors\nplt.title('2024 Olympics Medal Tally (Top 10 Countries)')\nplt.xlabel('Country')\nplt.ylabel('Medal Count')\nplt.xticks(rotation=45, ha='right')

----

{'text': ''}


----